# Домашнее задание 2 по теме: сравнение качества POS-теггеров и выделение синтаксических групп

In [126]:
import csv
from pymystem3 import Mystem
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag, map_tag
from pymorphy2 import MorphAnalyzer
from natasha import (Segmenter, NewsMorphTagger, NewsEmbedding, Doc, NewsSyntaxParser)
import spacy
from flair.models import SequenceTagger
from flair.data import Sentence
from sklearn.metrics import accuracy_score
import re

In [2]:
with open('rus_benchmark_plain.txt', encoding='utf-8') as fh:
    text = fh.read()

Этот текст подходит как сложный, так как это отрывки из стихотворений В. Хлебникова (привет словам типа "крылышкуя"), здесь есть омонимы типа "о" (междометие/предлог), "суша" (деепричатие/существительное), "жил" (глагол/существительное). Ещё здесь есть старорусский союз "аль" и странное звукоподражание "пинь".

In [3]:
print(text)

Крылышкуя золотописьмом
Тончайших жил,
Кузнечик в кузов пуза уложил
Прибрежных много трав и вер.
«Пинь, пинь, пинь!» — тарарахнул зинзивер.
О, лебедиво!
О, озари!
Гонимый — кем, почем я знаю?
Вопросом: поцелуев в жизни сколько?
Румынкой, дочерью Дуная,
Иль песнью лет про прелесть польки,—
Бегу в леса, ущелья, пропасти
И там живу сквозь птичий гам,
Как снежный сноп, сияют лопасти
Крыла, сверкавшего врагам.
Хотел бы шляхтичем на сейме,
Руку положив на рукоятку сабли,
Тому, отсвет желаний чей мы,
Крикнуть, чтоб узы воль ослабли.
Ты дичишься? что причина?
Аль не я рукой одною
Удержу на пашне тройку?
Аль не я спалил весною
Так, со зла, шабра постройку?
Костры горят сторожевые
На всех священных площадях,
И вижу — едут часовые
На челнах, лодках и конях.
О, если б волосами синих рек
Мне Азия покрыла бы колени
И дева прошептала таинственные пени,
И тихая, счастливая, рыдала,
Концом косы глаза суша.


In [4]:
benchmark = []
with open('rus_benchmark_pos.csv', encoding='utf-8') as fh:
    words_pos = csv.reader(fh, delimiter=';')
    for el in words_pos:
        benchmark.append(tuple(el))
print('всего слов:', len(benchmark))

всего слов: 144


## Русский язык

### Mystem

In [5]:
def mystem_convert(tag):
    if tag == 'ANUM' or tag == 'APRO':
        return 'A'
    elif tag == 'SPRO':
        return 'PRO'
    elif tag == 'ADVPRO':
        return 'ADV'
    else:
        return tag

In [6]:
def mystem_pos(text):
    m = Mystem()
    ana = m.analyze(text)
    mystem_variant = []
    for a in ana:
        if 'analysis' in a:
            pos = a['analysis'][0]['gr'].split(',')
            pos = pos[0]
            if '=' in pos:
                pos = pos.split('=')
                pos = pos[0]
            pos = mystem_convert(pos)
            word = a['text']
            wp = tuple([word, pos])
            mystem_variant.append(wp)
    print('всего слов в анализе Mystem:', len(mystem_variant))
    return mystem_variant

In [7]:
print('accuracy для Mystem:', 
      round(accuracy_score([y[1] for y in benchmark], 
                           [y[1] for y in mystem_pos(text)]), 4))

всего слов в анализе Mystem: 144
accuracy для Mystem: 0.9236


### Pymorhy

In [8]:
def pymorhy_convert(tag):
    tags = {'NOUN': 'S', 'ADJF': 'A', 'ADJS': 'A', 'COMP': 'A',
           'VERB': 'V', 'INFN': 'V', 'PRTF': 'V', 'PRTS': 'V', 'GRND': 'V',
           'NUMR': 'NUM', 'ADVB': 'ADV', 'NPRO': 'PRO', 'PREP': 'PR',
           'CONJ': 'CONJ', 'PRCL': 'PART', 'INTJ': 'INTJ'}
    return tags[tag]

In [9]:
def pymorphy_pos(text):
    words = [w.lower() for w in word_tokenize(text) if w.isalpha()]
    morph = MorphAnalyzer()
    analize_list = []
    for token in words:
        token_analized = morph.parse(token)[0]
        pos = str(token_analized.tag.POS)
        analize_list.append(tuple([token, pymorhy_convert(pos)]))
    return analize_list

In [10]:
print('accuracy для Pymorphy:', 
      round(accuracy_score([y[1] for y in benchmark], 
                           [y[1] for y in pymorphy_pos(text)]), 4))

accuracy для Pymorphy: 0.8958


### Natasha

In [11]:
def natasha_convert(tag):
    tags = {'ADJ': 'A', 'ADP': 'PR', 'ADV': 'ADV', 'AUX': 'V', 'CONJ': 'CONJ',
           'CCONJ': 'CONJ', 'DET': 'A', 'INTJ': 'INTJ', 'NOUN': 'S', 'NUM': 'NUM',
            'PART': 'PART', 'PROPN': 'S', 'PRON': 'PRO', 'SCONJ': 'CONJ', 
            'VERB': 'V'}
    return tags[tag]

In [12]:
def natasha_pos(text):
    segmenter = Segmenter()
    emb = NewsEmbedding()
    morph_tagger = NewsMorphTagger(emb)
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    natasha_ana = []
    for el in doc.tokens:
        if el.pos != 'PUNCT':
            natasha_ana.append(tuple([el.text, natasha_convert(el.pos)]))
    print('всего слов в анализе Natasha:', len(natasha_ana))
    return natasha_ana

In [13]:
print('accuracy для Natasha:', 
      round(accuracy_score([y[1] for y in benchmark], 
                           [y[1] for y in natasha_pos(text)]), 4))

всего слов в анализе Natasha: 144
accuracy для Natasha: 0.7778


Получилось, что для моего Золотого стандарта лучшим морфоанализатором русского языка является Mystem!

## Английский

Это тексты нескольких твитов Илона Маска. Здесь есть и классическая конверсия типа "resume", "offer" и других (очень много!). Присутствует сленг  (например, "booty"), сокращения (например, "lmk"), междометие "haha", а также множество имён собственных ("Tesla", "Britain").

In [63]:
with open('eng_benchmark_plain.txt', encoding='utf-8') as fh:
    eng_text = fh.read()
print(eng_text)

We must pass the great filter.
The gauntlet has been thrown down! 
The prophecy will be fulfilled. 
Will be less roomy with three vacuum rocket engines added.
Turn volume to eleven & play Powerglide in your Tesla.
Call of booty, great game.
We will teach you what’s known about the brain, which is not much.
If you feel Neuralink might have incorrectly overlooked your resume or declined to  make an offer, please lmk in comment below.
of course I still love you.
Turns out you can make anything fly haha.
Olde skoole analog synthesizer from ancient Britain.
Bureaucracy is inherently kafkaesque.
Thanks Tesla owners & investors! Love you!! We will work super hard to earn your trust & support.
Who controls the memes, controls the universe.


In [15]:
eng_benchmark = []
with open('eng_benchmark_pos.csv', encoding='utf-8') as fh:
    words_pos = csv.reader(fh, delimiter=';')
    for el in words_pos:
        eng_benchmark.append(tuple(el))
print('всего слов:', len(eng_benchmark))

всего слов: 124


### spaCy

In [47]:
def spacy_pos(eng_text):    
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(eng_text)
    spacy_ana = []
    for token in doc:
        if token.pos_ != 'PUNCT' and token.pos_ != 'SPACE' and token.text != '&':
            spacy_ana.append(tuple([token.text, token.pos_]))
    print('всего слов в анализе Spacy:', len(spacy_ana))
    return spacy_ana

In [61]:
print('accuracy для Spacy:', 
      round(accuracy_score([y[1] for y in eng_benchmark], 
                           [y[1] for y in spacy_pos(eng_text)]), 4))

всего слов в анализе Spacy: 124
accuracy для Spacy: 0.9032


### Flair

In [58]:
def flair_pos(eng_text):    
    sentence = Sentence(eng_text)
    tagger = SequenceTagger.load('upos')
    tagger.predict(sentence)
    flair_ana = sentence.to_tagged_string()
    flair_ana = flair_ana.split()
    counter = 0
    flair_var = []
    for el in flair_ana:
        if counter % 2 == 0:
            if flair_ana[counter+1] != '<PUNCT>' and el != '&':
                pos = flair_ana[counter+1]
                pos = re.sub(r'<(.+?)>', r'\1', pos)
                if pos == 'AUX':
                    pos = 'VERB'
                flair_var.append(tuple([el, pos]))
        counter += 1
    print('всего слов в анализе Flair:', len(flair_var))
    return flair_var

In [62]:
for_flair_benchmark = []
for ana in eng_benchmark:
    if ana[1] == 'AUX':
        for_flair_benchmark.append(tuple([ana[0], 'VERB']))
    else:
        for_flair_benchmark.append(ana)
print('accuracy для Flair:', 
      round(accuracy_score([y[1] for y in for_flair_benchmark], 
                           [y[1] for y in flair_pos(eng_text)]), 4))

2020-10-18 22:07:25,801 loading file /Users/romankazakov/.flair/models/en-pos-ontonotes-v0.4.pt
всего слов в анализе Flair: 124
accuracy для Flair: 0.9435


### NLTK

In [129]:
nltk_text = word_tokenize(eng_text)
nltk_ana = nltk.pos_tag(nltk_text)

Получилось, что для моего Золотого стандарта лучшим морфоанализатором английского языка является Flair!

## Синтаксические группы

In [130]:
def synt_groups(text):   
    segmenter = Segmenter()
    emb = NewsEmbedding()
    syntax_parser = NewsSyntaxParser(emb)
    doc = Doc(text)
    doc.segment(segmenter)
    doc.parse_syntax(syntax_parser)
    #print(doc.sents[0].text)
    syntax_list = []
    for el in doc.tokens:
        if el.rel != 'punct':
            syntax_list.append(tuple([el.text, el.id, el.head_id]))
    print(len(syntax_list))
    m = Mystem()
    ana = m.analyze(text)
    mystem_variant = []
    for a in ana:
        if 'analysis' in a:
            pos = a['analysis'][0]['gr'].split(',')
            pos = pos[0]
            if '=' in pos:
                pos = pos.split('=')
                pos = pos[0]
            pos = mystem_convert(pos)
            word = a['text']
            wp = tuple([word, pos])
            mystem_variant.append(wp)
    counter = 0
    collocs_1 = [] # ADV + V, типа "классно посидели"
    collocs_2 = [] # 'не' + V + S, типа "не понравилась еда"
    collocs_3 = [] # SPRO + V, типа "нас отравили", "нас угостили"
    for morph, synt in zip(mystem_variant, syntax_list):
        if counter != 0:
            if morph[1] == 'V' and mystem_variant[counter-1][1] == 'ADV' and synt[1] == syntax_list[counter-1][2]:
                c = mystem_variant[counter-1][0] + ' ' + morph[0]
                collocs_1.append(c) 
            elif morph[1] == 'V' and mystem_variant[counter-1][0] == 'не':
                if mystem_variant[counter+1][1] == 'S' and synt[1] == syntax_list[counter+1][2]:
                    c = 'не' + ' ' + morph[0] + ' ' + mystem_variant[counter+1][0]
                    collocs_2.append(c)
            elif morph[1] == 'V' and mystem_variant[counter-1][1] == 'SPRO' and synt[1] == syntax_list[counter-1][2]:
                c = mystem_variant[counter-1][0] + ' ' + morph[0]
                collocs_3.append(c) 
        counter += 1
    return collocs_1, collocs_2, collocs_3

Запустить на материале 1 ДЗ я не успел, но вот функция. Примеры лучше всего объясняют, почему нужны именно такие синтаксические группы.